<a href="https://colab.research.google.com/github/mehrn79/npl_wrod2wec/blob/main/NLP_2wec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word2wec implementation using skip-grams**

##**Preprocessing**

### **librarys for preprocessing**

In [21]:
pip install hazm

In [22]:
from __future__ import unicode_literals
from hazm import *
from keras.preprocessing import text
from keras.preprocessing.sequence import skipgrams

### **getting our corpes and stop words**

In [23]:
with open('/content/shams.txt') as f:
  lines = f.readlines()

with open('/content/stopwords.txt') as file:
  lines = file.readlines()
stopWord = [item.replace('\n',"") for item in lines]
  

### **cleaning our corpes by stop words**

In [24]:
words= []
for sent in lines :
   words.append(word_tokenize(sent))

corpes=[]
for wordSent in words :
  for word in wordSent :
    if word in stopWord :
      wordSent.remove(word)
  corpes.append(' '.join(wordSent))



### **tokenize our corpes**

In [25]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpes)

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

vocab_size = len(word2id) + 1 
embed_size = 100

sentencesID = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in corpes]


### **generate skip-grams**

In [26]:
skip_grams = [skipgrams(sentID, vocabulary_size=vocab_size, window_size=2) for sentID in sentencesID]
